In [1]:
import pandas as pd
import math

In [2]:
from datetime import datetime
date = datetime.fromtimestamp(1584148400	).date()
print(date)

2020-03-13


In [3]:
main = pd.read_excel("statePop.xlsx")
AL_data = pd.read_excel("States_data/AL_data.xlsx")

In [4]:
fips = pd.read_csv("fips/us-state-ansi-fips.csv")
fips.head()

,stname,st,stusps
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [5]:
R_0 = 2                                # Basic reproduction ratio
incubationDur = 6.4                    # Average incubation per
infectionDur = 7                       # Average dur of infection
dt = 1                                 # Time step
peopleSeen = 13.7                       # Avg number of people seen by average American

alpha = 1 - math.exp(-1/incubationDur) # Daily prob of individual becoming infectious
gamma = 1 - math.exp(-1/infectionDur)  # Daily prob an individual recovers
beta = R_0 * gamma                     # Scaled transmission rate

In [6]:
def numSus(t, pop):                          # Susceptible to Exposed func
    infProb = I[t] * peopleSeen * pop/S[t]
    return infProb * dt                      # How many people infected people have met with, weighted by population outdoors

def numInfected(t):
    if (t < 24):
        asymp = .4                           # (1-asymp) is prob of asympotomatic
    else: 
        asymp = .8
    return (1-asymp) * alpha * E[t] * dt     # daily prob of being infected after exposure * exposed pop

def numRecovery(t):
    return gamma * I[t] * dt                 # daily prob that an individual recovers * number of infected people

In [7]:
def sGrowth(t, pop): # Current pop (S[t]) - exposed
    return S[t] - numSus(t, pop)

def eGrowth(t, pop): # number of people who have been exposed (numSus) - number of people infected in last timestep
    return numSus(t, pop) - numInfected(t)

def iGrowth(t): # Infected previously + newly infected - recovered + growth in actual infected cases
    return I[t] + numInfected(t) - numRecovery(t) 

def rGrowth(t): # Recovered
    return R[t] + numRecovery(t)

In [8]:
def numInfectedLarge(t, modf):
    bound = 24 - modf * 3.5
    print(bound)
    if (t < bound):
        asymp = .4                           # (1-asymp) is prob of asympotomatic
    else: 
        asymp = .8
    return (1-asymp) * alpha * E[t] * dt 

def sGrowthLarge(t, pop): # Current pop (S[t]) - exposed
    return S[t] - numSus(t, pop)

def eGrowthLarge(t, pop, modf): # number of people who have been exposed (numSus) - number of people infected in last timestep
    return numSus(t, pop) - numInfectedLarge(t, modf)

def iGrowthLarge(t, modf): # Infected previously + newly infected - recovered + growth in actual infected cases
    return I[t] + numInfectedLarge(t, modf) - numRecovery(t) 

def rGrowthLarge(t): # Recovered
    return R[t] + numRecovery(t)

In [9]:
def numInfectedSmall(t, modf):
    bound = 24 - modf * 2.5
    print(bound)
    if (t < bound):
        asymp = .4                           # (1-asymp) is prob of asympotomatic
    else: 
        asymp = .8
    return (1-asymp) * alpha * E[t] * dt 

def sGrowthSmall(t, pop): # Current pop (S[t]) - exposed
    return S[t] - numSus(t, pop)

def eGrowthSmall(t, pop, modf): # number of people who have been exposed (numSus) - number of people infected in last timestep
    return numSus(t, pop) - numInfectedSmall(t, modf)

def iGrowthSmall(t, modf): # Infected previously + newly infected - recovered + growth in actual infected cases
    return I[t] + numInfectedSmall(t, modf) - numRecovery(t) 

def rGrowthSmall(t): # Recovered
    return R[t] + numRecovery(t)

In [12]:
stData = pd.read_excel("States_data/AL_data.xlsx")
S = [main['Population'].iloc[0]]
print(S)
E = [0]
I = [0]
R = [0]
stData.tail()

[4903185]


,seconds_since_Epoch,tested,positive,deaths
195,1587504827,48387.0,5269,180.0
196,1587598378,48387.0,5589,196.0
197,1587620702,48387.0,5610,201.0
198,1587687036,48387.0,5832,201.0
199,1587762002,52641.0,5832,201.0


In [13]:
num = 24 + 21
firstCase = False
timeIndex=0
for j in range(num):
    if j < 2:
        datePath = '3-0' + str(j + 8)
    elif 2 <= j < 24:
        datePath = '3-' + str(j + 8)
    elif 24 <= j < 33:
        datePath = '4-0' + str(j - 23)
    else:
        datePath = '4-' + str(j - 23)
    dateNow = datePath + '-2020'
    ourDate = datetime.strptime(dateNow, '%m-%d-%Y').date()

    if not firstCase:
        infectedNum = 0
        for x in range(timeIndex, stData.shape[0]):
            covidDate = stData['seconds_since_Epoch'].iloc[x]
            covidDate = datetime.fromtimestamp(covidDate).date()
            if ourDate == covidDate:
                infectedNum = stData.iloc[j][2]
                if not math.isnan(infectedNum) and int(infectedNum) != 0:
                    firstCase = True
                    I[j] += int(infectedNum)
                    break
            elif covidDate > ourDate:
                timeIndex = x
                break

    path = "data/data" + datePath + '.xlsx'
    data = pd.read_excel(path)
    percentAtHome = data.loc[j]['percentAtHome']
    atHome = percentAtHome / 100 * S[j]
    
    #if main.loc[j] > 10000000
    if False:
        modf = main.loc[45]['Population'] / 10000000
        print(modf)
        S.append(sGrowthLarge(j, S[j] - atHome))
        E.append(eGrowthLarge(j, S[j] - atHome, modf))
        I.append(iGrowthLarge(j, modf))
        R.append(rGrowthLarge(j))
    #elif main.loc[j] < 2000000:
    elif False:
        modf = (main.loc[50]['Population'] / 200000)
        print(modf)
        S.append(sGrowthSmall(j, S[j] - atHome))
        E.append(eGrowthSmall(j, S[j] - atHome, modf))
        I.append(iGrowthSmall(j, modf))
        R.append(rGrowthSmall(j))
    else:
        print(1)
        S.append(sGrowth(j, S[j] - atHome))
        E.append(eGrowth(j, S[j] - atHome))
        I.append(iGrowth(j))
        R.append(rGrowth(j))
    print("Infected", end="")
    print(I)
    print("\n")

1
Infected[0, 0.0]


1
Infected[0, 0.0, 0.0]


1
Infected[0, 0.0, 1.0, 0.8668778997501816]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237, 6.89499874167478]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237, 6.89499874167478, 9.57243407016036]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237, 6.89499874167478, 9.57243407016036, 13.387576881395852]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 

1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237, 6.89499874167478, 9.57243407016036, 13.387576881395852, 19.401398865485216, 27.37256790657104, 39.3332100896525, 55.829016004679886, 72.55393739885504, 99.36823327302014, 140.0611557722975, 201.27182640766466, 285.92526100660757, 403.84107729910687, 566.809748024478, 725.0305549003313, 933.5030998408035, 1263.4954266230307, 1308.6790037414191, 1483.966615568867, 1600.484878996675, 1746.5559098060876, 1897.926247365332, 1970.2785911358687, 2135.542739195043, 2224.185068119629]


1
Infected[0, 0.0, 1.0, 0.8668778997501816, 1.6580385616530382, 2.2230680682984283, 3.367887480502495, 4.874927414785237, 6.89499874167478, 9.57243407016036, 13.387576881395852, 19.401398865485216, 27.37256790657104, 39.3332100896525, 55.829016004679886, 72.55393739885504, 99.36823327302014, 140.0611557722975, 201.27182640766466, 285.92526100660757, 403.84107729910687, 566.809748024478, 725.0

In [382]:
stDatas = pd.read_excel("States_data/AL_data.xlsx")
stDatas

,seconds_since_Epoch,tested,positive,deaths
0,1583651326,NaN,0,NaN
1,1583900198,10.0,0,NaN
2,1584110139,12.0,1,NaN
3,1584148400,28.0,5,0.0
4,1584161186,28.0,6,0.0
...,...,...,...,...
195,1587504827,48387.0,5269,180.0
196,1587598378,48387.0,5589,196.0
197,1587620702,48387.0,5610,201.0
198,1587687036,48387.0,5832,201.0
